<a href="https://colab.research.google.com/github/MrWou/retrive-Flights-data-from-Assoaeroporti-Italy/blob/main/download_flights_passengers_statistics_for_ITA_Airports.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

##Download Flights Passengers Satistics for Italian Airports
Source: [Official Italian Airport Association](https://assaeroporti.com/)

In [ ]:
import pandas as pd
import requests
from io import BytesIO
import time
import random
from requests.exceptions import RequestException
import numpy as np

def generate_excel_urls(start_year, start_month, end_year, end_month):
    """Genera una lista di URL per i file Excel mensili."""
    urls = []
    for year in range(start_year, end_year + 1):
        for month in range(1, 13):
            if year == start_year and month < start_month:
                continue
            if year == end_year and month > end_month:
                break
            month_str = str(month).zfill(2)
            url = f"https://data-upload.assaeroporti.com/download-export/{year}/{month_str}"
            urls.append(url)
    return urls

def download_and_process_excel(url, max_retries=3, initial_delay=1, max_delay=10):
    """Scarica un file Excel, legge il foglio 'Passeggeri Mese' e lo processa, con retry."""
    for attempt in range(max_retries):
        try:
            response = requests.get(url, timeout=20)
            response.raise_for_status()  # Solleva un'eccezione per errori HTTP

            excel_file = BytesIO(response.content)
            df = pd.read_excel(excel_file, sheet_name="Passeggeri Mese", header=1) # use second row as header

            # Estrai anno e mese dall'URL
            year = int(url.split("/")[-2])
            month = int(url.split("/")[-1])

            # Crea la colonna 'data'
            df['data'] = f"{year}-{month:02}-01"

            # Seleziona e rinomina le colonne
            df_selected = df[[
                "data",
                "Aeroporto",
                "Nazionali",
                "Internazionali",
                "Di cui UE",
                "Transiti Diretti",
                "Aviazione Generale e altri"
            ]].copy()

            df_selected.rename(columns={
                    "Nazionali": "passeggeri_nazionali",
                    "Internazionali": "passeggeri_internazionali",
                    "Di cui UE": "passeggeri_ue",
                    "Transiti Diretti": "transiti_diretti",
                    "Aviazione Generale e altri": "aviazione_generale_e_altri",
                }, inplace=True
            )
            # Calcola la colonna "internazionali"
            df_selected["passeggeri_internazionali"] = df_selected["passeggeri_internazionali"] - df_selected["passeggeri_ue"]

            return df_selected

        except RequestException as e:
            print(f"Errore nel download di {url}, tentativo {attempt + 1}/{max_retries}: {e}")
            if attempt == max_retries - 1:
                print(f"Impossibile scaricare {url} dopo {max_retries} tentativi.")
                return None
            delay = min(initial_delay * (2 ** attempt) + random.uniform(0, 1), max_delay)
            print(f"Riprovo dopo {delay:.2f} secondi.")
            time.sleep(delay)
        except Exception as e:
            print(f"Errore nella lettura o elaborazione del file da {url}: {e}")
            return None

def main():
    """Funzione principale per eseguire il download e l'elaborazione dei dati."""
    start_year, start_month = 2019, 1
    end_year, end_month = 2024, 10
    urls = generate_excel_urls(start_year, start_month, end_year, end_month)

    all_data = []
    for i, url in enumerate(urls):

        df = download_and_process_excel(url)
        if df is not None:
            all_data.append(df)
        delay_between_requests = 1 # pause between request
        print(f"Processing {i + 1} of {len(urls)}, waiting {delay_between_requests} second")
        time.sleep(delay_between_requests)

    if all_data:
            combined_df = pd.concat(all_data, ignore_index=True)

            # Gestione dei NaN
            numerical_cols = combined_df.select_dtypes(include=np.number).columns
            combined_df[numerical_cols] = combined_df[numerical_cols].fillna(0)


            # Filtra righe con "TOTALI" o NaN nel campo Aeroporto
            combined_df = combined_df[~combined_df["Aeroporto"].isin(["TOTALI", np.nan])]

            # Rimuovi "(*)" e "(^)" dai nomi degli aeroporti
            combined_df["Aeroporto"] = combined_df["Aeroporto"].str.replace(r"(\(\*\)|\(\^\))", "", regex=True)

            # Trasformazione in formato long
            id_vars = ["data", "Aeroporto"]
            value_vars = [
                "passeggeri_nazionali",
                "passeggeri_internazionali",
                "passeggeri_ue",
                "transiti_diretti",
                "aviazione_generale_e_altri"
            ]
            long_df = pd.melt(combined_df, id_vars=id_vars, value_vars=value_vars, var_name="tipologia_passeggeri", value_name="valore")

            print(f"DataFrame creato con successo:\n{long_df.head()}")
            print(f"Shape del dataframe: {long_df.shape}")
            print("Salvataggio del file csv")
            long_df.to_csv("passeggeri_aeroporti_long.csv", index=False)
    else:
        print("Nessun dato è stato scaricato o processato.")

if __name__ == "__main__":
    main()

Processing 1 of 70, waiting 1 second
Processing 2 of 70, waiting 1 second
Processing 3 of 70, waiting 1 second
Processing 4 of 70, waiting 1 second
Processing 5 of 70, waiting 1 second
Processing 6 of 70, waiting 1 second
Processing 7 of 70, waiting 1 second
Processing 8 of 70, waiting 1 second
Processing 9 of 70, waiting 1 second
Processing 10 of 70, waiting 1 second
Processing 11 of 70, waiting 1 second
Processing 12 of 70, waiting 1 second
Processing 13 of 70, waiting 1 second
Processing 14 of 70, waiting 1 second
Processing 15 of 70, waiting 1 second
Processing 16 of 70, waiting 1 second
Processing 17 of 70, waiting 1 second
Processing 18 of 70, waiting 1 second
Processing 19 of 70, waiting 1 second
Processing 20 of 70, waiting 1 second
Processing 21 of 70, waiting 1 second
Processing 22 of 70, waiting 1 second
Processing 23 of 70, waiting 1 second
Processing 24 of 70, waiting 1 second
Processing 25 of 70, waiting 1 second
Processing 26 of 70, waiting 1 second
Processing 27 of 70, 